# Считывание данных

In [297]:
import pandas as pd
import numpy as np

In [298]:
data = pd.read_csv('data/data.csv')
data.head()

,qn1,qn2,qn3,qn4,qn5,qn6,qn7,qn8,qn9,qn10,qn11,target
0,NaN,0.496372,1.356517,NaN,0.201783,47.104283,2.017825e-01,1.191783,3.513918e+07,0.365426,1.624946,0
1,247.899905,NaN,NaN,2.009905,NaN,NaN,1.223077e+10,3.219905,1.211625e+07,NaN,NaN,0
2,NaN,-1.168700,-1.292688,NaN,-1.292688,356.428482,-1.292688e+00,-0.012688,1.516536e+05,0.088284,-0.151629,0
3,13.669695,0.669695,NaN,-0.315305,-1.330305,NaN,1.170006e+08,0.339695,5.068618e+05,-0.330305,NaN,0
4,102.404032,-0.118875,12.493512,0.252032,-0.595860,79.846382,1.512900e+10,3.304032,1.080490e+05,-0.541999,-0.107430,0


Посмотрим на данные

In [299]:
data.target.value_counts()

target
0    972
1     28
Name: count, dtype: int64

In [300]:
data.isna().sum()

qn1       587
qn2        73
qn3        80
qn4       587
qn5        73
qn6        84
qn7         0
qn8       164
qn9         0
qn10       74
qn11       86
target      0
dtype: int64

# Однофакторный анализ

## Availability

In [301]:
# считаем долю не NaN значений
availability = data.notna().sum() / data.shape[0]
availability

qn1       0.413
qn2       0.927
qn3       0.920
qn4       0.413
qn5       0.927
qn6       0.916
qn7       1.000
qn8       0.836
qn9       1.000
qn10      0.926
qn11      0.914
target    1.000
dtype: float64

In [302]:
# выбираем те признаки, у которых доля Availability >= 0.8
availability[availability >= 0.8]

qn2       0.927
qn3       0.920
qn5       0.927
qn6       0.916
qn7       1.000
qn8       0.836
qn9       1.000
qn10      0.926
qn11      0.914
target    1.000
dtype: float64

In [303]:
# оставляем в данных только те признаки, у которых availability >= 80%
data = data[availability[availability >= 0.8].index]
data.head()

,qn2,qn3,qn5,qn6,qn7,qn8,qn9,qn10,qn11,target
0,0.496372,1.356517,0.201783,47.104283,2.017825e-01,1.191783,3.513918e+07,0.365426,1.624946,0
1,NaN,NaN,NaN,NaN,1.223077e+10,3.219905,1.211625e+07,NaN,NaN,0
2,-1.168700,-1.292688,-1.292688,356.428482,-1.292688e+00,-0.012688,1.516536e+05,0.088284,-0.151629,0
3,0.669695,NaN,-1.330305,NaN,1.170006e+08,0.339695,5.068618e+05,-0.330305,NaN,0
4,-0.118875,12.493512,-0.595860,79.846382,1.512900e+10,3.304032,1.080490e+05,-0.541999,-0.107430,0


## WoE преобразование

Воспользуемся библиотекой scorecardpy для WoE преобразования

In [304]:
import scorecardpy as sc

In [305]:
# отбросим признаки, которые имеют iv меньше 0.02, то есть малоинформативные признаки

data_iv = sc.var_filter(data, y="target", iv_limit=0.02)
data_iv.head()

[INFO] filtering variables ...


,qn3,qn6,qn5,qn11,qn8,qn2,qn10,target
0,1.356517,47.104283,0.201783,1.624946,1.191783,0.496372,0.365426,0
1,NaN,NaN,NaN,NaN,3.219905,NaN,NaN,0
2,-1.292688,356.428482,-1.292688,-0.151629,-0.012688,-1.168700,0.088284,0
3,NaN,NaN,-1.330305,NaN,0.339695,0.669695,-0.330305,0
4,12.493512,79.846382,-0.595860,-0.107430,3.304032,-0.118875,-0.541999,0


In [306]:
# выполним WoE преобразование

bins = sc.woebin(data_iv, y="target")
data_woe = sc.woebin_ply(data_iv, bins)
data_woe.head()

[INFO] creating woe binning ...


/Users/kudr.max/PycharmProjects/company-default-prediction/shaders/venv/lib/python3.11/site-packages/scorecardpy/condition_fun.py:40: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  datetime_cols = dat.apply(pd.to_numeric,errors='ignore').select_dtypes(object).apply(pd.to_datetime,errors='ignore').select_dtypes('datetime64').columns.tolist()
/Users/kudr.max/PycharmProjects/company-default-prediction/shaders/venv/lib/python3.11/site-packages/scorecardpy/condition_fun.py:40: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  datetime_cols = dat.apply(pd.to_numeric,errors='ignore').select_dtypes(object).apply(pd.to_datetime,errors='ignore').select_dtypes('datetime64').columns.tolist()
/Users/kudr.max/PycharmProjects/company-default-prediction/shaders/venv/lib/pytho

[INFO] converting into woe values ...


/Users/kudr.max/PycharmProjects/company-default-prediction/shaders/venv/lib/python3.11/site-packages/scorecardpy/woebin.py:361: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  init_bin = init_bin.groupby('brkp', group_keys=False).agg({
/Users/kudr.max/PycharmProjects/company-default-prediction/shaders/venv/lib/python3.11/site-packages/scorecardpy/woebin.py:414: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  .agg({'good':sum, 'bad':sum}).reset_index()\
/Users/kudr.max/PycharmProjects/company-default-prediction/shaders/venv/lib/python3.11/site-packages/scorecardpy/woebin.py:414: FutureWarning: The provided callable <built-i

,target,qn3_woe,qn5_woe,qn11_woe,qn6_woe,qn8_woe,qn2_woe,qn10_woe
0,0,-0.022381,0.148218,0.050644,-1.723281,-0.634899,0.139635,-0.381395
1,0,-0.822297,-0.880306,-0.895500,-0.871689,-0.344669,-0.880306,-0.893911
2,0,1.070613,-0.073545,-0.001028,0.701412,-0.182550,0.034274,0.470992
3,0,-0.822297,-0.073545,-0.895500,-0.871689,-0.634899,0.139635,1.281922
4,0,-0.924487,-0.073545,-0.001028,-1.723281,-0.344669,0.327098,-0.903326


# Многофакторный анализ

### Отбор признаков через L1-регуляризацию

In [307]:
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

X = data_woe.drop(columns=['target']).to_numpy()
y = data_woe.target.to_numpy()

k_fold = StratifiedKFold(n_splits=5, shuffle=True)
scaler = StandardScaler()
model_l1 = LogisticRegression(penalty='l1', solver='liblinear')

gini_best = -1
C_best = None
coef_best = None
for C in [0.001, 0.01, 0.1, 1, 10, 100, 1000]:
    model_l1.C = C
    ginis = []

    for i, (train_index, test_index) in enumerate(k_fold.split(X, y)):

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        model_l1.fit(X_train, y_train)
        y_pred = model_l1.predict(X_test)

        gini = 2 * roc_auc_score(y_test, y_pred) - 1
        ginis.append(gini)

    gini_mean = np.mean(ginis)
    if gini_mean > gini_best:
        gini_best = gini_mean
        C_best = C
        coef_best = model_l1.coef_

In [308]:
gini_best

0.14666666666666667

In [309]:
model_l1.coef_

array([[0.68906588, 0.32325461, 0.68874486, 0.74478997, 0.6323109 ,
        0.29218895, 0.32442691]])

Видно, что ни один из коэффициентов не занулился, значит мы не будеи выкидывать никакие признаки из рассмотрения

### Обучение модели с L2-регуляризацией

In [310]:
k_fold = StratifiedKFold(n_splits=5, shuffle=True)
scaler = StandardScaler()
model_l2 = LogisticRegression(penalty='l2', solver='liblinear')

gini_best = -1
C_best = None
coef_best = None
for C in [0.001, 0.01, 0.1, 1, 10, 100, 1000]:
    model_l2.C = C
    ginis = []

    for i, (train_index, test_index) in enumerate(k_fold.split(X, y)):

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        model_l2.fit(X_train, y_train)
        y_pred = model_l2.predict(X_test)

        gini = roc_auc_score(y_test, y_pred)
        ginis.append(gini)

    gini_mean = np.mean(ginis)
    if gini_mean > gini_best:
        gini_best = gini_mean
        C_best = C
        coef_best = model_l2.coef_

### Итоговая модель

In [322]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = LogisticRegression(penalty='l2', solver='liblinear', C=C_best)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
roc_auc = roc_auc_score(y_test, y_pred)
gini = 2 * roc_auc - 1

In [323]:
gini

0.2727272727272727

In [324]:
roc_auc

0.6363636363636364